In [1]:
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import mysql.connector
import secrets
import string
import random
import getpass
import zlib

def generate_IV():
    return get_random_bytes(AES.block_size)

def generate_key(password):
    salt = get_random_bytes(32)
    key = PBKDF2(password, salt, 32)
    return key

def generate_password():
    lowercase = list(string.ascii_lowercase)
    uppercase = list(string.ascii_uppercase)
    num = list(string.digits)
    specialchar = ["!", "@", "#", "$", "%", "^", "&", "*"]
    pswd = lowercase + uppercase + num + specialchar
    random.shuffle(pswd)
    length = random.randint(8, 14)
    password = []
    for i in range(length):
        password.append(secrets.choice(pswd))
        random.shuffle(pswd)

    mandate = True
    ualpha = lalpha = number = specialCharacter = 0

    while mandate:
        for char in password:
            if 'A' <= char <= 'Z':
                ualpha += 1
            elif 'a' <= char <= 'z':
                lalpha += 1
            elif '0' <= char <= '9':
                number += 1
            else:
                specialCharacter += 1

            if ualpha != 0 and lalpha != 0 and number != 0 and specialCharacter != 0:
                mandate = False
            else:
                if ualpha == 0:
                    password[random.randint(0, len(password) - 1)] = secrets.choice(uppercase)
                elif lalpha == 0:
                    password[random.randint(0, len(password) - 1)] = secrets.choice(lowercase)
                elif number == 0:
                    password[random.randint(0, len(password) - 1)] = secrets.choice(num)
                elif specialCharacter == 0:
                    password[random.randint(0, len(password) - 1)] = secrets.choice(specialchar)

    return "".join(password)

def encrypt_data(key, iv, data):
    # Compress the data before encryption
    compressed_data = zlib.compress(data.encode('utf-8'))
    
    cipher = AES.new(key, AES.MODE_CBC, iv)
    cipher_data = cipher.encrypt(pad(compressed_data, AES.block_size, style='pkcs7'))

    return cipher_data

def decrypt_data(key, iv, cipher_data):
    cipher = AES.new(key, AES.MODE_CBC, iv)
    compressed_data = unpad(cipher.decrypt(cipher_data), AES.block_size, style='pkcs7')

    # Decompress the data after decryption
    decompressed_data = zlib.decompress(compressed_data)

    return decompressed_data.decode('utf-8') if decompressed_data else None

def fetch_key(username):
    db = mysql.connector.connect(host="localhost", user="root", password="Happier", database="passwordmanager")
    cur = db.cursor()

    t = (username,)
    s = """SELECT key1 FROM iv_n_key WHERE Username = %s"""
    cur.execute(s, t)
    result = cur.fetchone()

    if result:
        return result[0]

    cur.close()
    db.close()
    return None

def save_to_database(website, username, password):
    db = mysql.connector.connect(host="localhost", user="root", password="Happier", database="passwordmanager")
    cur = db.cursor()

    key = fetch_key(username)
    if key is None:
        key = generate_key("sk231108@")
        t_key = (website, username, key)
        s_key = """INSERT INTO iv_n_key (Website,Username, key1) VALUES (%s,%s, %s)"""
        cur.execute(s_key, t_key)
        db.commit()

    key = fetch_key(username)
    iv = generate_IV()

    # Convert the password to bytes before encryption
    password_bytes = password.encode('utf-8') if isinstance(password, str) else password
    cipher_data = encrypt_data(key, iv, password_bytes)

    # Convert the cipher_data to a string for storage in the database
    cipher_str = cipher_data.decode('latin-1')

    s_psswrd = """INSERT INTO psswrd_manager (Website, Username, Password, IV) VALUES (%s, %s, %s, %s)"""
    t_psswrd = (website, username, cipher_str, iv)
    cur.execute(s_psswrd, t_psswrd)
    db.commit()

    cur.close()
    db.close()


def get_password_from_database(website, username):
    db = mysql.connector.connect(host="localhost", user="root", password="Happier", database="passwordmanager")
    cur = db.cursor()

    key = fetch_key(username)
    if key is None:
        print("Key not found for the user.")
        return None

    t = (website, username)
    s = """SELECT Password, IV FROM psswrd_manager WHERE Website = %s AND Username = %s"""
    cur.execute(s, t)
    result = cur.fetchone()

    if result:
        encrypted_password, iv = result
        decrypted_password = decrypt_data(key, iv, encrypted_password)
        print(f"Decrypted Password: {decrypted_password}")
        return decrypted_password

    cur.close()
    db.close()
    return None


def update_password(website, username, new_password):
    db = mysql.connector.connect(host="localhost", user="root", password="Happier", database="passwordmanager")
    cur = db.cursor()

    key = fetch_key(username)
    if key is None:
        print("Key not found for the user.")
        return

    t = (website, username)
    s = """SELECT IV FROM psswrd_manager WHERE Website = %s AND Username = %s"""
    cur.execute(s, t)
    iv = cur.fetchone()

    if iv:
        iv = iv[0]
        cipher_data = encrypt_data(key, iv, new_password)

        # Update the password entry with the new encrypted password
        t_psswrd = (cipher_data, website, username)
        s_psswrd = """UPDATE psswrd_manager SET Password = %s WHERE Website = %s AND Username = %s"""
        cur.execute(s_psswrd, t_psswrd)
        db.commit()

        print("Password updated successfully.")
    else:
        print("Password entry not found.")

    cur.close()
    db.close()


def delete_password(website, username):
    db = mysql.connector.connect(host="localhost", user="root", password="Happier", database="passwordmanager")
    cur = db.cursor()

    # Delete the password entry
    t_psswrd = (website, username)
    s_psswrd = """DELETE FROM psswrd_manager WHERE Website = %s AND Username = %s"""
    cur.execute(s_psswrd, t_psswrd)
    db.commit()

    # Delete the key entry
    t_key = (website, username)
    s_key = """DELETE FROM iv_n_key WHERE Website = %s AND Username = %s"""
    cur.execute(s_key, t_key)
    db.commit()

    cur.close()
    db.close()



def user_interface():
    print("Enter 1 to save a new password")
    print("Enter 2 to generate a password")
    print("Enter 3 to show your saved password")
    print("Enter 4 to update your password")
    print("Enter 5 to delete a password")
#     print("Enter 4 to delete a password")
    print("Enter 0 to exit")

    choice = int(input())
    while choice != 0:
        if choice == 1:
            website = input("Enter website: ")
            username = input("Enter username: ")
            password_choice = int(input("Enter 1 to input your password or 2 to generate one: "))
            if password_choice == 1:
                # Use getpass to securely input the password
                password = getpass.getpass("Enter your password: ")
            elif password_choice == 2:
                password = generate_password()
                print(f"Generated Password: {password}")

            save_to_database(website, username, password)

        elif choice == 2:
            password = generate_password()
            print(f"Generated Password: {password}")

        elif choice == 3:
            website = input("Enter the website: ")
            username = input("Enter the username: ")
            get_password_from_database(website, username)
        elif choice == 5:
            website = input("Enter the website: ")
            username = input("Enter the username: ")
            delete_password(website, username)

#         # elif choice == 4:
#         #     website = input("Enter the website: ")
#         #     username = input("Enter the username: ")
#         #     new_password_choice = int(input("Enter 1 to input your new password or 2 to generate one: "))
#         #     if new_password_choice == 1:
#         #         new_password = input("Enter your new password: ")
#         #     elif new_password_choice == 2:
#         #         new_password = generate_password()
#         #         print(f"Generated Password: {new_password}")
#         #     update_password(website, username, new_password)

#         # elif choice == 5:
#         #     pass

#         elif choice == 4:
#             website = input("Enter the website: ")
#             username = input("Enter the username: ")
#             delete_password(website, username)
        

    # Add the new choice for updating a password
        elif choice == 4:
            website = input("Enter the website: ")
            username = input("Enter the username: ")
            new_password_choice = int(input("Enter 1 to input your new password or 2 to generate one: "))
            if new_password_choice == 1:
                new_password = getpass.getpass("Enter your new password: ")
            elif new_password_choice == 2:
                new_password = generate_password()
                print(f"Generated Password: {new_password}")
            update_password(website, username, new_password)


        else:
            print("Invalid choice. Please try again.")

        print("\nEnter 0 to exit or choose another option.")
        choice = int(input())

user_interface()



Enter 1 to save a new password
Enter 2 to generate a password
Enter 3 to show your saved password
Enter 4 to update your password
Enter 5 to delete a password
Enter 0 to exit
0
